In [ ]:
#6-2 se arreglo un bug en las fechas recibidas. 
#Se modifico la funcion periodo_str_short en la linea marcada y se agrego errors='coerce' en cierre , adjudicacion y publicacion para manejar fechas erroneas
import awswrangler as wr
import pandas as pd


In [ ]:
dfOriginal = wr.s3.read_parquet(path='s3://cchc-dw-qa-raw/mercado_publico/licitaciones_finales/', dataset=True)

In [ ]:
DATE_FORMAT = "%d-%m-%Y"

_MESES_SHORT = [
    "ene",
    "feb",
    "mar",
    "abr",
    "may",
    "jun",
    "jul",
    "ago",
    "sep",
    "oct",
    "nov",
    "dic",
]

def periodo_str_short(periodo):
    if periodo is None:
        return None
    #*
    if periodo != periodo:
        return None
    #*
    if len(periodo) == 5:
        mes = int(periodo[4:5])
        yr = periodo[0:4]

        str_mes = _MESES_SHORT[mes - 1]
    else:
        mes = int(periodo[4:6])
        yr = periodo[0:4]

        str_mes = _MESES_SHORT[mes - 1]

    return f"{str_mes} - {yr}"

In [ ]:
df = dfOriginal.copy()

df.columns = df.columns.str.lower()

to_convert = [
    "cantidad_adjudicada",
    "presupuesto",
    "total_adjudicado",
    "precio_unitario",
]
for x in to_convert:
    df[x] = df[x].str.replace(".", "", regex=True)
    df[x] = df[x].str.replace(",", ".", regex=True)
    df[x] = pd.to_numeric(df[x], errors="coerce")
    df[x] = df[x].round(0).astype("Int64")

df.cierre = df.cierre.str.split(" ").str[0]
df.cierre = pd.to_datetime(df.cierre, format=DATE_FORMAT, errors="coerce")

df.publicacion = df.publicacion.str.split(" ").str[0]
df.publicacion = pd.to_datetime(df.publicacion, format=DATE_FORMAT, errors="coerce")

df.adjudicacion = df.adjudicacion.str.split(" ").str[0]
df.adjudicacion = pd.to_datetime(
    df.adjudicacion, format=DATE_FORMAT, errors="coerce"
)

to_capitalize = [
    "lic_nombre",
    "lic_descrip",
    "stenderclientfullname",
    "fullname",
    "competidor",
]
for x in to_capitalize:
    df[x] = df[x].str.capitalize()

df["periodo_num"] = df.adjudicacion.dt.strftime("%Y%m")
# Por alcance se agregan estas dos columnas, que no necesariamente se utilizan
df["periodo_str"] = df.periodo_num.apply(lambda x: periodo_str_short(x))
df["periodo"] = df.periodo_num.apply(lambda x: periodo_str_short(x))

response = wr.s3.to_parquet(
        df=df,
        path="s3://cchc-dw-dev-staging/mercado_publico/licitaciones_finales/",
        dataset=True,
        database="staging_dev",
        mode="overwrite",
        table="mercado_publico_licitaciones_finales",
        compression="gzip",
        dtype={"cierre": "date", "publicacion": "date", "adjudicacion": "date"},
    )



In [ ]:
df = df[df.adjudicacion.isnull()]
df

In [ ]:
test = pd.to_datetime('2020-07-24', format=DATE_FORMAT)
test

In [ ]:
df = dfOriginal.copy()
#df = df[(df.RUT == '94.557.000-8') & df.TenderCode == '5407-122-O122']
df = df.iloc[364873]
df